In [42]:
import os, re, json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
import spacy
from langdetect import detect
from transformers import pipeline
from keybert import KeyBERT
from sklearn.cluster import KMeans
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from collections import Counter

In [ ]:
# Télécharger les ressources NLTK
nltk.download("stopwords", quiet=True)
nltk.download("punkt", quiet=True)
nltk.download("vader_lexicon", quiet=True)

True

# 1) Fusion & inspection rapide

**Objectif** : charger les deux CSV et créer un seul DataFrame maître avec une colonne source.

In [43]:
oms = pd.read_csv("../outputs/articles_oms.csv")      
forbes = pd.read_csv("../outputs/forbes_articles.csv")

oms['source'] = 'OMS'
forbes['source'] = 'Forbes'
df = pd.concat([oms, forbes], ignore_index=True)

# aperçu
print(df.shape)
print(df.columns.tolist())
df.head()

(65, 5)
['source', 'titre', 'date', 'lien', 'texte']


,source,titre,date,lien,texte
0,OMS,L’OMS publie des orientations pour faire face ...,3 novembre 2025,https://www.who.int/fr/news/item/03-11-2025-wh...,L’Organisation mondiale de la Santé (OMS) publ...
1,OMS,L’OMS publie un guide mondial pour des société...,31 octobre 2025,https://www.who.int/fr/news/item/31-10-2025-wh...,À l’occasion de la Journée mondiale des villes...
2,OMS,Lancement d’un cours en ligne sur l’utilisatio...,30 octobre 2025,https://www.who.int/fr/news/item/30-10-2025-la...,À l’occasion du Mois de la sensibilisation au ...
3,OMS,L’OMS condamne le massacre de patients et de c...,29 octobre 2025,https://www.who.int/fr/news/item/29-10-2025-wh...,L’Organisation mondiale de la Santé (OMS) cond...
4,OMS,"Selon un nouveau rapport du Lancet Countdown, ...",29 octobre 2025,https://www.who.int/fr/news/item/29-10-2025-cl...,Alors qu’un nouveau rapport mondial publié auj...


In [3]:
df.to_csv("../data/all_articles.csv", index=False)

# 2) Nettoyage de base (HTML, espaces, encodage, dates)

**Objectif** : normaliser le texte brut.

In [19]:
# === Nettoyage léger (pour BERT) et prétraitement pour TF-IDF ===


nlp_fr = spacy.load("fr_core_news_sm", disable=["parser","ner"])
nlp_en = spacy.load("en_core_web_sm", disable=["parser","ner"])

def clean_html_only(text):
    """Nettoyage léger : retire script/style/URLs, conserve phrases intactes."""
    soup = BeautifulSoup(str(text), "html.parser")
    for s in soup(["script","style"]):
        s.decompose()
    out = soup.get_text(separator=" ")
    out = re.sub(r"https?://\S+|www\.\S+|\S+@\S+", " ", out)
    out = out.replace("’", "'").replace("‘","'")
    out = re.sub(r"\s+", " ", out).strip()
    txt = re.sub(r'javascript.*disabled.*', ' ', text, flags=re.I)
    txt = re.sub(r'cookie.*', ' ', text, flags=re.I)
    return out

def preprocess_for_tfidf(text, lang_hint='fr', min_tok=2):
    """Nettoyage plus agressif pour TF-IDF / LDA : lowercase, remove stopwords, lemmatisation."""
    if not isinstance(text, str):
        text = str(text or "")
    text = clean_html_only(text)
    text = text.lower()
    # keep letters & accents & apostrophes and spaces
    text = re.sub(r"[^a-z0-9àâäçéèêëîïôöùûüÿœæ'\s-]", " ", text)
    doc = (nlp_fr if str(lang_hint).startswith("fr") else nlp_en)(text)
    toks = []
    for t in doc:
        if t.is_stop or t.is_punct or t.is_space or t.like_num:
            continue
        lemma = t.lemma_.lower().strip()
        if len(lemma) < min_tok:
            continue
        toks.append(lemma)
    return " ".join(toks)

# detection de langue

def safe_detect(s):
    try: return detect(s)
    except: return 'unknown'


In [20]:
df_clean = df.copy()


df_clean['texte_clean_bert'] = df['texte'].astype(str).apply(clean_html_only)
df_clean['lang'] = df_clean['texte_clean_bert'].apply(lambda s: safe_detect(s) if s.strip() else 'unknown')
df_clean['texte_clean_tfidf'] = df_clean.apply(lambda r: preprocess_for_tfidf(r['texte'], lang_hint=r['lang']), axis=1)

# vérifier quelques exemples
df_clean[['texte','texte_clean_bert','texte_clean_tfidf']].head(3)

,texte,texte_clean_bert,texte_clean_tfidf
0,L’Organisation mondiale de la Santé (OMS) publ...,L'Organisation mondiale de la Santé (OMS) publ...,organisation mondial santé oms publier aujourd...
1,À l’occasion de la Journée mondiale des villes...,À l'occasion de la Journée mondiale des villes...,occasion journée mondial ville organisation mo...
2,À l’occasion du Mois de la sensibilisation au ...,À l'occasion du Mois de la sensibilisation au ...,occasion mois sensibilisation cancer sein cent...


In [ ]:
before = ' '.join(df_clean['texte_clean_bert'].astype(str).tolist()).split()
after = ' '.join(df_clean['texte_clean_tfidf'].astype(str).tolist()).split()
Counter(before).most_common(30)

[('de', 2195),
 ('et', 1192),
 ('la', 1074),
 ('des', 884),
 ('les', 821),
 ('à', 777),
 ('le', 578),
 ('en', 573),
 ('du', 431),
 ('pour', 342),
 ('un', 330),
 ('dans', 320),
 ('une', 246),
 ('sur', 229),
 ('plus', 228),
 ('que', 213),
 ('est', 212),
 ('aux', 210),
 ('qui', 193),
 ('santé', 193),
 ('au', 184),
 ('a', 180),
 ('par', 157),
 ('«', 149),
 (':', 139),
 ('avec', 126),
 ('sont', 117),
 ('pays', 108),
 ('ou', 101),
 ('Le', 91)]

In [24]:
Counter(after).most_common(30)

[('santé', 252),
 ('pays', 145),
 ('om', 130),
 ('mondial', 99),
 ('numérique', 82),
 ('africain', 82),
 ('être', 79),
 ('afrique', 78),
 ('développement', 69),
 ('national', 67),
 ('service', 66),
 ('international', 66),
 ('secteur', 63),
 ('système', 61),
 ('nouveau', 58),
 ('monde', 58),
 ('stratégique', 55),
 ('renforcer', 55),
 ('projet', 54),
 ('local', 53),
 ("aujourd'hui", 52),
 ('an', 52),
 ('million', 49),
 ('personne', 48),
 ('soin', 47),
 ('continent', 47),
 ('femme', 45),
 ('entreprise', 45),
 ('investissement', 44),
 ('faire', 43)]

# 3) Déduplication & filtrage

**Objectif** : enlever doublons exacts et presque-identiques.

In [26]:
# doublons exacts sur le texte
df_clean = df_clean.drop_duplicates(subset=['texte']).reset_index(drop=True)

# 5) Prétraitement linguistique (tokenize / stopwords / lemmatisation)

**Objectif** : préparer texte pour modèles classiques (TF-IDF, LDA) et pour embeddings.

# 6) Représentation numérique

A) **TF-IDF** (rapide, interprétable) — pour classification, clustering léger, recherche.

In [28]:
tfidf = TfidfVectorizer(max_features=15000, ngram_range=(1,2))
X_tfidf = tfidf.fit_transform(df_clean['texte_clean_tfidf'].fillna(''))

B) **Embeddings** (BERT / SentenceTransformers) — mieux pour clustering sémantique, topics modernes, similarity.

In [30]:
# pip install sentence-transformers
model = SentenceTransformer('all-mpnet-base-v2')
embeddings = model.encode(df_clean['texte_clean_bert'].tolist(), show_progress_bar=True)

Batches: 100%|██████████| 2/2 [00:22<00:00, 11.19s/it]


# 7) Topic Modeling

* Si TF-IDF -> **NMF** (souvent meilleur) ou **LDA** (gensim).
* Si embeddings -> **BERTopic** ou clustering + keywords extraction.

In [31]:
texts = [t.split() for t in df_clean['texte_clean_tfidf']]
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(t) for t in texts]
lda = gensim.models.LdaModel(corpus, id2word=dictionary, num_topics=8, passes=10, random_state=42)

for i,topic in lda.print_topics(-1):
    print(i, topic)

0 0.011*"om" + 0.009*"santé" + 0.008*"produit" + 0.007*"être" + 0.007*"public" + 0.006*"trachome" + 0.005*"autorité" + 0.005*"fidji" + 0.004*"sanitaire" + 0.004*"cancer"
1 0.007*"cookie" + 0.006*"site" + 0.006*"agricole" + 0.006*"être" + 0.005*"risque" + 0.005*"afrique" + 0.004*"terminal" + 0.004*"hpp" + 0.004*"femme" + 0.004*"pays"
2 0.009*"africain" + 0.007*"local" + 0.007*"afrique" + 0.006*"million" + 0.006*"tabac" + 0.006*"mondial" + 0.006*"numérique" + 0.005*"banque" + 0.005*"financement" + 0.005*"sfi"
3 0.009*"développement" + 0.009*"gabonais" + 0.009*"pays" + 0.006*"gabon" + 0.006*"tourisme" + 0.005*"projet" + 0.005*"afrique" + 0.005*"secteur" + 0.005*"national" + 0.004*"durable"
4 0.032*"santé" + 0.015*"om" + 0.012*"pays" + 0.008*"mondial" + 0.007*"système" + 0.006*"personne" + 0.006*"soin" + 0.005*"maladie" + 0.004*"numérique" + 0.004*"cours"
5 0.012*"numérique" + 0.007*"africain" + 0.007*"cap-vert" + 0.007*"secteur" + 0.006*"dangote" + 0.006*"pays" + 0.006*"développement" + 0

# 8) Clustering d’articles

KMeans sur TF-IDF ou HDBSCAN sur embeddings (si grand jeu).

In [32]:
kmeans = KMeans(n_clusters=8).fit(X_tfidf)
df_clean['cluster_tfidf'] = kmeans.predict(X_tfidf)

# k = 8
# km = KMeans(n_clusters=k, random_state=42)
# clusters = km.fit_predict(X_tfidf)
# df['cluster'] = clusters

In [34]:
# HDBSCAN on embeddings
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=5).fit(embeddings)
df_clean['cluster_embed'] = clusterer.labels_

# clusterer = hdbscan.HDBSCAN(min_cluster_size=5)
# df['cluster_emb'] = clusterer.fit_predict(embeddings)

In [ ]:
import umap.umap_ as umap
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='cosine', random_state=42)
umap_tf = reducer.fit_transform(X_tfidf)
umap_emb = reducer.fit_transform(embeddings)


AttributeError: module 'umap' has no attribute 'UMAP'

# 9) Extraction de mots-clé / résumés / phrases clés

* Keywords : RAKE / YAKE / KeyBERT.
* Résumé : transformers (Bart, T5) — extractive vs abstractive.

In [37]:


kw = KeyBERT(model='all-mpnet-base-v2')
df_clean['keywords'] = df_clean['texte_clean_bert'].apply(lambda t: kw.extract_keywords(t, top_n=5))


In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
df['summary'] = df['text'].apply(lambda t: summarizer(t, max_length=60, min_length=20)[0]['summary_text'])

# 10) Sentiment & ton / polarité

* Pour l’anglais: modèles HuggingFace (sst2, cardiffnlp).
* Pour le français: CamemBERT fine-tuned (ou modèle multilingual).

In [38]:
sent = pipeline('sentiment-analysis')
df_clean['sentiment'] = df_clean['texte_clean_bert'].apply(lambda t: sent(t[:512])[0])

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
d:\Mes_dossiers\IDSI\M2\Course\Text_Mining\textmining-sante-innovation\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ulric\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on

# 11) NER (entités nommées)

In [ ]:
def extract_entities(text, lang='fr'):
    nlp = nlp_fr if lang=='fr' else nlp_en
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

df['entities'] = df.apply(lambda r: extract_entities(r['text'], r['lang']), axis=1)

# 12) Analyse comparative *OMS vs Forbes*

**Objectifs d’analyse** :

* Distribution des topics par source.
* Mots-clés distinctifs (chi2 ou log-likelihood) par source.
* Sentiment moyen par source.
* Entités les plus citées par source.

Exemple : mots discriminants via chi2 (scikit-learn)

In [39]:
from sklearn.feature_selection import chi2
y = (df_clean['source']=='OMS').astype(int)
chi2scores, p = chi2(X_tfidf, y)
top_n = 30
terms = tfidf.get_feature_names_out()
top_terms = [terms[i] for i in chi2scores.argsort()[-top_n:][::-1]]
top_terms

['santé',
 'om',
 'soin',
 'système',
 'sanitaire',
 'maladie',
 'personne',
 'mental',
 'système santé',
 'tabac',
 'pacifique',
 'santé mental',
 'cours',
 'cancer',
 'résistance',
 'neurologique',
 'enfant',
 'apprentissage',
 'vaccin',
 'trachome',
 'el fasher',
 'fasher',
 'infection',
 'rapport',
 'santé numérique',
 'fidji',
 'surveillance',
 'hpp',
 'mondial',
 'health']

# 13) Visualisation & reporting

* Wordclouds par source.
* t-SNE / UMAP des embeddings pour voir clusters.
* Barplots des topics, entités, sentiment.

Exemple UMAP + scatter:

In [40]:
import umap
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='cosine', random_state=42)
emb_2d = reducer.fit_transform(embeddings)
df['x'], df['y'] = emb_2d[:,0], emb_2d[:,1]

AttributeError: module 'umap' has no attribute 'UMAP'

# 14) Export des résultats

Sauvegarder DataFrame enrichi :

In [41]:
df_clean.to_csv("../data/all_articles_processed.csv", index=False)

## Récapitulatif rapide (ordre d’exécution conseillé)

1. Fusion + inspection
2. Nettoyage HTML + normalisation
3. Déduplication + détection langue
4. Prétraitement spaCy (lemmatisation, stopwords)
5. Vecteurs (TF-IDF et/ou embeddings)
6. Topic modelling + clustering
7. NER + keywords + résumé + sentiment
8. Analyses comparatives OMS vs Forbes
9. Visualisations & export
10. QA manuelle et itérations